In [0]:
from keras.models import Model
from keras.layers import Input, merge, ZeroPadding2D
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import keras.backend as K
from keras.models import Sequential
from keras.layers.merge import concatenate ,Concatenate
global  feat_map,k,gr_rate,num_dense
global inp_shape,num_densem,inp_shape,compres,layers,num_class

feat_map= 32
inp_shp=(224,224,3)
compres=0.5 
k=16
gr_rate=feat_map
layers_in_b = [6,12,24,16]
num_dense=4
num_class=1000

In [0]:
#model.add(Lambda(lambda x: x *MyValue))

In [0]:
def int_blocks():
  org_inp=Input(shape=inp_shp)

 
  inp=ZeroPadding2D((3,3))(org_inp)
  inp=Convolution2D(k, 7, strides=2)(inp)  #dense net B is of standard filters 16 
  inp=BatchNormalization(axis=3)  (inp)
  inp=Activation(activation='relu')(inp)
  inp=ZeroPadding2D((1,1))(inp)
  inp=MaxPooling2D((3, 3), strides=2)(inp)
  for dense in range(num_dense - 1):
      
      inp = dense_block(inp, layers_in_b[dense])


      inp = trans_block(inp)
#final layer in dense 

 
  inp = dense_block(inp, layers_in_b[len(layers_in_b)-1])

  inp = BatchNormalization()(inp)
  
  inp = Activation('relu')(inp)
  inp = GlobalAveragePooling2D()(inp)
  inp   =Dense(num_class, activation='softmax')(inp)


  model = Model(org_inp,inp)

  

  return model  
 

In [0]:
def trans_block(inp):
  inp= BatchNormalization( axis=3)(inp)
  inp=Activation(activation='relu')(inp)
  inp=Convolution2D(int(k * compres), 1, 1)(inp)# default compresion =1,
  inp= AveragePooling2D((2, 2), strides=2)(inp)
   
  return inp

In [0]:
def conv_block(inp,k_conv):
  #1*1 covilution 
  channels=4*k_conv
  inp=BatchNormalization( axis=3)(inp)
  inp   =Activation('relu')(inp)
  inp= Convolution2D(channels, 1)(inp)
  
  
  #3*3 covilution 
  inp=BatchNormalization( axis=3 )(inp)   
  inp   =Activation('relu')(inp)
  
  inp=ZeroPadding2D((1, 1))(inp)
  inp=Convolution2D(k_conv, 3)(inp)
   
  return inp            

In [0]:
def dense_block(inp,num_layers):
  conc_inp = inp
  for l in range(num_layers):
        
        inp = conv_block(conc_inp, gr_rate)
        conc_inp = Concatenate(axis=3)([conc_inp,inp])



  return conc_inp
    
    

In [109]:
 

model=int_blocks()
  
print(model.summary())   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (1, 1))`
  after removing the cwd from sys.path.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_504 (ZeroPadding (None, 230, 230, 3)  0           input_21[0][0]                   
__________________________________________________________________________________________________
conv2d_977 (Conv2D)             (None, 112, 112, 16) 2368        zero_padding2d_504[0][0]         
__________________________________________________________________________________________________
batch_normalization_993 (BatchN (None, 112, 112, 16) 64          conv2d_977[0][0]                 
__________________________________________________________________________________________________
activation